In [2]:
import numpy as np
import pandas as pd
import os

# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# from scipy.special import rel_entr
# from scipy.stats import entropy
# import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [3]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
! nvidia-smi

Sun Jun 12 15:11:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 30%   38C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 28%   

# MongoDB

In [5]:
from pymongo import MongoClient
import pymongo

In [6]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2011 = db.rm_patent_cn_2011
    db_rm_us_2011 = db.rm_patent_us_2011
    db_rm_de_2011 = db.rm_patent_de_2011

    
except errors.ConnectionFailure as err:
    print(err)

# 2011

## Company數量

In [7]:
data_assignee = db_rm_de_2011.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [8]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [9]:
data_assignee = db_rm_cn_2011.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
data_assignee = db_rm_us_2011.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [11]:
final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [12]:
# 取總數前30家公司
final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)

,company,count
182,"Jinhu Zhongbo IOT Technology Co., Ltd.",30
173,Jiangsu IoT Research and Development Center,17
266,Qualcomm Inc,16
402,Xerox Corp,13
201,LG Electronics Inc,12
419,ZTE Corp,9
120,"Gaoxing Zhilian Technology Co., Ltd.",9
299,Samsung Electronics Co Ltd,8
75,China core Microelectronics Technology Chengdu...,7
361,Telefonaktiebolaget LM Ericsson AB,7


In [13]:
com_df = final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)
com_df.reset_index(drop=True, inplace=True)

In [14]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,"Jinhu Zhongbo IOT Technology Co., Ltd.",30,3.401197
1,Jiangsu IoT Research and Development Center,17,2.833213
2,Qualcomm Inc,16,2.772589
3,Xerox Corp,13,2.564949
4,LG Electronics Inc,12,2.484907
5,ZTE Corp,9,2.197225
6,"Gaoxing Zhilian Technology Co., Ltd.",9,2.197225
7,Samsung Electronics Co Ltd,8,2.079442
8,China core Microelectronics Technology Chengdu...,7,1.945910
9,Telefonaktiebolaget LM Ericsson AB,7,1.945910


公司名稱正規化

In [15]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Co. Ltd.","")\
                                                    .replace(" Co., Ltd.","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("..","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba")\
                                                    .replace("ZTE Intelligent IoT Technology","ZTE"))

In [16]:
com_df

,company,count,count_log
0,Jinhu Zhongbo IOT Technology,30,3.401197
1,Jiangsu IoT Research and Development Center,17,2.833213
2,Qualcomm,16,2.772589
3,Xerox,13,2.564949
4,LG Electronics,12,2.484907
5,ZTE,9,2.197225
6,Gaoxing Zhilian Technology,9,2.197225
7,Samsung Electronics,8,2.079442
8,China core Microelectronics Technology Chengdu...,7,1.945910
9,Telefonaktiebolaget LM Ericsson AB,7,1.945910


In [17]:
com_ls = list(com_df.company)

# Data 2011

富比士2000資料

In [18]:
f_data = pd.read_csv("./Forbes/Forbes_2011.csv")

In [19]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Industry,Country,Market Value,Sales,Profits,Assets,Rank,Forbes Webpage,Profits as % of Assets,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
32,Samsung Electronics,Semiconductors,South Korea,112.9,133.800,13.7000,119.3,33,http://www.forbes.com/companies/samsung-electr...,0.114837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,Intel,Semiconductors,United States,114.5,43.600,11.5000,63.2,93,http://www.forbes.com/companies/intel,0.181962,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,Taiwan Semiconductor,Semiconductors,Taiwan,61.3,13.300,5.1000,23.7,273,http://www.forbes.com/companies/taiwan-semicon...,0.215190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,Texas Instruments,Semiconductors,United States,40.3,14.000,3.2000,13.4,396,http://www.forbes.com/companies/texas-instruments,0.238806,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547,Applied Materials,Semiconductors,United States,19.8,10.400,1.4000,11.3,548,http://www.forbes.com/companies/applied-materials,0.123894,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,Micron Technology,Semiconductors,United States,10.2,9.000,1.8000,14.6,603,http://www.forbes.com/companies/micron-technology,0.123288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658,STMicroelectronics,Semiconductors,Switzerland,11.0,10.500,0.8417,13.0,658,http://www.forbes.com/companies/stmicroelectro...,0.064746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,Broadcom,Semiconductors,United States,21.6,6.800,1.1000,7.9,736,http://www.forbes.com/companies/broadcom,0.139241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
751,ASML Holding,Semiconductors,Netherlands,17.8,6.000,1.4000,8.2,752,http://www.forbes.com/companies/asml-holding,0.170732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1030,Hynix Semiconductor,Semiconductors,South Korea,14.5,6.800,-0.2984,13.8,1031,http://www.forbes.com/companies/hynix-semicond...,-0.021623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
Ind_ls = list(f_data[f_data["Industry"]=="Semiconductors"].Company)

## 使用company top 30 做join

In [21]:
forb_ls = list(np.where(com_df.company.isin(Ind_ls), 1, 0))

In [22]:
com_df[com_df.company.isin(Ind_ls)]

,company,count,count_log
7,Samsung Electronics,8,2.079442


In [25]:
# pbc 0.1, 0.1
comp_ls = ["Jiangsu IoT Research and Development Center","Xerox","Datang Mobile Communications Equipment","IoT Holdings","LG Electronics","Qualcomm","Telefonaktiebolaget LM Ericsson AB","Xerox","ZTE"]

In [26]:
competitor_ls = list(np.where(com_df.company.isin(comp_ls), 1, 0))

In [27]:
com_df[com_df.company.isin(comp_ls)]

,company,count,count_log
1,Jiangsu IoT Research and Development Center,17,2.833213
2,Qualcomm,16,2.772589
3,Xerox,13,2.564949
4,LG Electronics,12,2.484907
5,ZTE,9,2.197225
9,Telefonaktiebolaget LM Ericsson AB,7,1.945910
10,Datang Mobile Communications Equipment,6,1.791759
12,IoT Holdings,6,1.791759


In [28]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.02904990722043802
ari:  -0.04113110539845758


## 使用Forbes做join

In [29]:
forb_ls = list(np.where(f_data.Company.isin(Ind_ls), 1, 0))

In [30]:
# pbc 0.1, 0.1
comp_ls = ["Jiangsu IoT Research and Development Center","Xerox","Datang Mobile Communications Equipment","IoT Holdings","LG Electronics","Qualcomm","Telefonaktiebolaget LM Ericsson AB","Xerox","ZTE"]

In [31]:
competitor_ls = list(np.where(f_data.Company.isin(comp_ls), 1, 0))

In [32]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.000656746782454566
ari:  -0.0034893828444070153
